In [3]:
import json

# Read and inspect the JSON file
with open('C:\\Users\\djsma\\Downloads\\inbox\\somaanrauniyar_955361995856784\\message_1.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Print the structure of the data
print(json.dumps(data, indent=4))


{
    "participants": [
        {
            "name": "somaan rauniyar"
        },
        {
            "name": "\u00f0\u009d\u0099\u00b3\u00f0\u009d\u009a\u0092\u00f0\u009d\u009a\u009f\u00f0\u009d\u009a\u0093\u00f0\u009d\u009a\u0098\u00f0\u009d\u009a\u009d \u00f0\u009d\u0099\u00bc\u00f0\u009d\u009a\u008a\u00f0\u009d\u009a\u0097\u00f0\u009d\u009a\u008c\u00f0\u009d\u009a\u0091\u00f0\u009d\u009a\u008a\u00f0\u009d\u009a\u0097\u00f0\u009d\u009a\u008d\u00f0\u009d\u009a\u008a"
        }
    ],
    "messages": [
        {
            "sender_name": "somaan rauniyar",
            "timestamp_ms": 1703742608219,
            "content": "\u00e0\u00a4\u00ae\u00e0\u00a5\u0085\u00e0\u00a4\u00b8\u00e0\u00a5\u0087\u00e0\u00a4\u009c \u00e0\u00a4\u00b2\u00e0\u00a4\u00be\u00e0\u00a4\u0087\u00e0\u00a4\u0095 \u00e0\u00a4\u0095\u00e0\u00a5\u0087\u00e0\u00a4\u00b2\u00e0\u00a4\u00be",
            "is_geoblocked_for_viewer": false
        },
        {
            "sender_name": "\u00f0\u009d\u0099\u00b3\u00f0\

In [6]:
import json
import pandas as pd
from pandas import json_normalize

# Load the JSON file
file_path = 'C:\\Users\\djsma\\Downloads\\inbox\\somaanrauniyar_955361995856784\\message_1.json'
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Normalize the 'participants' part
participants_df = json_normalize(data['participants'])
print("Participants DataFrame:")
print(participants_df.head())

# Normalize the 'messages' part, including nested 'reactions' if present
messages_df = json_normalize(data['messages'], 'reactions', ['sender_name', 'timestamp_ms', 'content', 'is_geoblocked_for_viewer'], errors='ignore')
print("Messages DataFrame:")
print(messages_df.head())


Participants DataFrame:
                                                name
0                                    somaan rauniyar
1  ð³ððððð ð¼ððððð...


KeyError: "Key 'reactions' not found. If specifying a record_path, all elements of data should have the path."

In [5]:
from pandas import json_normalize

# Normalize the JSON data
df = json_normalize(data)
print(df.head())


                                        participants  \
0  [{'name': 'somaan rauniyar'}, {'name': 'ð³ð...   

                                            messages            title  \
0  [{'sender_name': 'somaan rauniyar', 'timestamp...  somaan rauniyar   

   is_still_participant                           thread_path magic_words  
0                  True  inbox/somaanrauniyar_955361995856784          []  


In [7]:
!pip install unidecode


In [1]:
import os
import json
import pandas as pd
import re
from unidecode import unidecode

# Specify the directory containing the JSON files
json_directory = 'C:\\Users\\djsma\\Downloads\\inbox'

def normalize_text(text):
    # Remove non-ASCII characters and convert fancy characters to closest ASCII equivalents
    return unidecode(re.sub(r'[^\x00-\x7F]+', '', text))

def decode_unicode(obj):
    if isinstance(obj, str):
        return normalize_text(obj)
    return obj

def process_json_file(file_path, sender_name):
    # Read the JSON file
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file, object_hook=decode_unicode)

    # Extract the messages from the data
    messages = data['messages']

    # Create a DataFrame from the messages
    df = pd.DataFrame(messages)

    # Add a unique identifier column based on the index
    df.insert(0, 'message_id', range(1, len(df) + 1))

    # Add the sender name as a new column
    df['sender_name'] = sender_name

    # Ensure 'content' field exists in each message
    df['content'] = df.get('content', '')

    return df

# Initialize an empty list to store the DataFrames
df_list = []

# Iterate over all subdirectories in the main directory
for sender_name in os.listdir(json_directory):
    account_dir = os.path.join(json_directory, sender_name)
    if os.path.isdir(account_dir):
        json_file = os.path.join(account_dir, 'message_1.json')
        if os.path.isfile(json_file):
            df = process_json_file(json_file, sender_name)
            df_list.append(df)

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(df_list, ignore_index=True)

# Print the final DataFrame
print(final_df)


        message_id                     sender_name   timestamp_ms  \
0                1                1026181925445709  1703736018137   
1                2                1026181925445709  1703706444233   
2                3                1026181925445709  1703706444021   
3                4                1026181925445709  1703701591987   
4                5                1026181925445709  1686472657738   
...            ...                             ...            ...   
134450           6  _samrithi_12__1097959504930365  1677670470963   
134451           7  _samrithi_12__1097959504930365  1677670410115   
134452           1    __garvesh___1038626274197022  1671279391787   
134453           2    __garvesh___1038626274197022  1671278030694   
134454           3    __garvesh___1038626274197022  1671278013741   

                                                  content  \
0               à¤®à¥à¤¸à¥à¤ à¤²à¤¾à¤à¤ à¤à¥à¤²à¤¾   
1       guywhoclicksss wasn't notified about this

In [1]:
import os
import json
import pandas as pd
import emoji

# Specify the directory containing the JSON files
json_directory = 'C:\\Users\\djsma\\Downloads\\inbox'

def process_json_files(json_files, folder_name):
    messages = []
    
    for file_path in sorted(json_files, key=lambda x: int(x.split('_')[-1].split('.')[0])):
        # Read the JSON file
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            messages.extend(data['messages'])
    
    # Check the number of unique senders
    senders = {msg.get('sender_name') for msg in messages if 'sender_name' in msg}
    if len(senders) != 2:
        return None
    
    # Create a DataFrame from the messages
    df = pd.DataFrame(messages)
    
    # Sort messages by timestamp in ascending order
    df.sort_values(by='timestamp_ms', inplace=True)
    
    # Function to decode messages
    def decode_message(content):
        if pd.isna(content):
            return ""
        try:
            content = content.encode('latin1').decode('unicode-escape')
        except UnicodeDecodeError:
            content = content.encode('utf-8').decode('utf-8')
        return emoji.demojize(content)
    
    # Apply the decode_message function to the 'content' field
    df['message'] = df.apply(
        lambda x: decode_message(x['content']) if 'content' in x and isinstance(x['content'], str) else "", axis=1
    )
    
    # Add a column for share (yes/no)
    df['share'] = df.apply(
        lambda x: 'yes' if 'share' in x and isinstance(x['share'], dict) else 'no', axis=1
    )
    
    # Add a column for reactions
    df['reactions'] = df.apply(
        lambda x: emoji.demojize(x['reactions'][0]['reaction']) if 'reactions' in x and isinstance(x['reactions'], list) and len(x['reactions']) > 0 else pd.NA, axis=1
    )
    
    # Combine messages from the same sender within 2 minutes
    combined_messages = []
    last_sender = None
    last_timestamp = None
    combined_message = ""
    share = "no"
    reactions = pd.NA
    
    for index, row in df.iterrows():
        sender = row['sender_name']
        timestamp = row['timestamp_ms']
        
        if last_sender == sender and (timestamp - last_timestamp) <= 120000:
            combined_message += "\n\n" + row['message']
        else:
            if combined_message:
                combined_messages.append((last_sender, last_timestamp, combined_message.strip(), share, reactions))
            combined_message = row['message']
            last_sender = sender
            last_timestamp = timestamp
            share = row['share']
            reactions = row['reactions']
    
    if combined_message:
        combined_messages.append((last_sender, last_timestamp, combined_message.strip(), share, reactions))
    
    # Create a new DataFrame with combined messages
    combined_df = pd.DataFrame(combined_messages, columns=['sender_name', 'timestamp_ms', 'message', 'share', 'reactions'])
    combined_df['message_id'] = range(1, len(combined_df) + 1)
    combined_df['folder_name'] = folder_name
    
    return combined_df

# Initialize an empty list to store the DataFrames
df_list = []

# Iterate over all subdirectories in the main directory
for folder_name in os.listdir(json_directory):
    account_dir = os.path.join(json_directory, folder_name)
    if os.path.isdir(account_dir):
        json_files = [os.path.join(account_dir, file) for file in os.listdir(account_dir) if file.startswith('message_') and file.endswith('.json')]
        if json_files:
            df = process_json_files(json_files, folder_name)
            if df is not None:
                df_list.append(df)

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(df_list, ignore_index=True)

# Select specific columns and rename them
final_df = final_df[['message_id', 'folder_name', 'sender_name', 'timestamp_ms', 'message', 'share', 'reactions']]
final_df.rename(columns={'timestamp_ms': 'timestamp'}, inplace=True)

# Save the DataFrame to a CSV file
output_file = 'instagram_messages1.csv'
final_df.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"DataFrame saved to {output_file}")


DataFrame saved to instagram_messages1.csv
